In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
real_estate_valuation = fetch_ucirepo(id=477)

# data (as pandas dataframes)
X = real_estate_valuation.data.features
y = real_estate_valuation.data.targets

X.drop(columns=['No'], inplace=True)

# metadata
print(real_estate_valuation.metadata)

# variable information
print(real_estate_valuation.variables)


ModuleNotFoundError: No module named 'ucimlrepo'

In [1]:
X.head()

NameError: name 'X' is not defined